# Define parent sequence class, later inherit class in a DNASeq and ProSeq class

 ### Sequence Class notes
    1.Sequence class mainly bulit from previous code
    2. provide a simple translation dictionary to X-reference inside object (may not be the most efficient route,might be hidden redunancy)

### DNASeq Class notes:
    1.DNASeq class dep
    requirements
        -a translate method that will convert the DNA sequence and return a ProteinSequence object
        -one other method of your choice (what you did previously is fine)
    2. DNA seq should check that it is in fact a DNA seq, functionality checking validity

### ProteinSeq Class notes:
            -a method of your choice. In this case, if the method you would implement is too complex to reasonably implement or would use resources you don't have access to, it is okay to leave it as what is called a stub method (has only one line, "pass") and explain in comments what this method would do and it's purpose
            a: ProtienSeq must mave different elements than DNASeq but similarties to Sequence class for inheir
            b: What might be unique to protien sequences that might be useful data to add to a method?

In [1]:
import datetime as dt#a way to snapshot current datetime

In [2]:
def readMultilineFASTA(file):

	fh = open(file)

	name = None
	seq = ""


	for line in fh:
		if line.startswith(">"):   #Dr. Whaley supplied fasta file pull in function
			if name:
				
				yield (name,seq)
				name = None
				seq = ""

			name = line.rstrip().lstrip(">")

		else:
			seq += line.rstrip()
			

	yield name,seq

#### parent class 

In [3]:
class Sequence:
    #supplied DNA translation 
    transDict = {'M':['ATG'], 'F':['TTT', 'TTC'], 'L':['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], 'C':['TGT', 'TGC'], 'Y':['TAC', 'TAT'], 'W':['TGG'], 'P':['CCT', 'CCC', 'CCA', 'CCG'], 'H':['CAT', 'CAC'],
    'Q':['CAA', 'CAG'], 'R':['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'I':['ATT', 'ATC', 'ATA'], 'T':['ACT', 'ACC', 'ACA', 'ACG'],
    'N':['AAT', 'AAC'], 'K':['AAA', 'AAG'], 'S':['AGT', 'AGC', 'TCT', 'TCC', 'TCA', 'TCG'], 'V':['GTT', 'GTC', 'GTA', 'GTG'],
    'A':['GCT', 'GCC', 'GCA', 'GCG'], 'D':['GAT', 'GAC'], 'E':['GAA', 'GAG'], 'G':['GGT', 'GGC', 'GGA', 'GGG'], '*':['TAA','TAG','TGA']}
    def __init__(self,stamp,seq): #initalize parent class Sequence
        self.seq = seq
        self.stamp = stamp # date time stamp of Seq creation
        self.orgLen = len(seq)  # this stores original char len, if seq's length is modifed then this will tell us that 
    def __repr__(self):
        return list(self.seq)

#### child class 1 of 2  (DNAsequence) checks for validity and provides a conversion to Protein Sequence
specific string return, a triple char list return, translation to Protein Sequence 

In [4]:
class DNASeq(Sequence): # Child sequence class to harness DNA sequence record
    
    
    
    def __init__(self,stamp,seq,seqHeader,label,index):
          
        if isinstance(seq, ProSeq):
            return "Protein Sequence not allowed..."
    
        else:
            DNAbase=['G','A','T','C','N'] #List to test valid DNAseq
            failindex=0
            for i in range(len(seq)):
                if seq[i] not in DNAbase:  #Test bases for DNA validity
                    failindex = i
            if failindex==0:
                super().__init__(stamp,seq)
                self.label = label
                self.seqHeader = seqHeader
                self.importindex = index
                self.orgLen=len(seq)
            else:
                print("Not a DNA SEQUENCE, last failed at char index "+str(failindex)+": "+str(seq[failindex]))

               
    def returnTriples(self):  #breaks DNA sequence into 3 character item list
        triples = [self.seq[i:i+3] for i in range(0, len(self.seq), 3)]
        return triples
    
    def transToPro(self):# a dictionary lookup for Value --translation method to ProSeq
        newSeq=""
        for i in self.returnTriples():
            for key, val in super().transDict.items():
                if i in val:
                    newSeq+=str(key)
        return newSeq
    
    def convertfromVal(self): #another converting method but transToPro I thought might be more efficient to demo
        newSeq=""
        testSTR =""
        for i in range(len(self.seq)):
            if i%3 ==0:
                if i ==0:
                    testSTR = self.seq[0:3]
                else:
                    testSTR = self.seq[i-3:i]
                
                for key, val in super().transDict.items():
                    if testSTR in val:
                        newSeq+=str(key)
                myindex = i
            else:
                myindex=i
        return newSeq
    
    def __repr__(self):
        if self.orgLen == len(self.seq):  # Checks if original Char length or if Modified
            retstr ="I'm DNASequence: "+self.label+"\n"
            retstr+="DataSequence length:" +str(self.orgLen)+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        else:
            retstr ="I'm DNASequence: "+self.label+"\n"
            retstr+="DataSequence orignal length:" +str(self.orgLen)+" but was modified to new Length:"+str(len(seq))+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq +"\n-------------------------------------------------------------"
        return retstr
                

#### child class 2 of 2 (Protein Sequence) allows linking from converted DNA sequence
specific string return, a set method, a convert return values from dictionary

In [5]:
class ProSeq(Sequence): #Child sequence class to harness DNA sequence record

    def __init__(self,stamp,seq,label):
        
        if isinstance(seq, DNASeq): #test if  DNASeq obj
            print("Not a valid Protien Sequence")
        else:
            failindex = 0
            for i in seq:           #check for valid Amino Acid Chars
                if i not in super().transDict.keys():
                        failindex+=1
            if failindex > 0:
                print(str(failindex)+ " invalid characters made this sequence not a proper protien sequence")
            else:
                
                super().__init__(stamp,seq)
                self.label = label
                self.DNAimportbit = False # a boolean set false later toggled by method linking to DNAseq
                self.DNAseqlabel=""
                self.importindex = 0
    
    def setFromDNA(self,genLabel,index):  #method to update generating sequence label and toggle import bit
        self.DNAseqlabel = genLabel
        self.DNAimportbit =True
        self.importindex = index
        
    def convertfromKEY(self):# a dictionary lookup from KEY
        newSeq=""
        for i in self.seq:
            for key,val in super().transDict.items():
                if i in key:
                    newSeq +=val[0]
        return newSeq
    
    def countAcids(self):# a simple method to count Amino Acid codes
        acidCount={}
        for i in self.seq:
            if i not in acidCount.keys():
                acidCount[i]=1
            else:
                acidCount[i]=int(acidCount[i])+1
        return acidCount
    
    def __repr__(self):
        retstr ="I'm Protein Sequence: "+self.label+"\n"
        if self.DNAimportbit == False:
            retstr+="DataSequence length:" +str(len(self.seq))+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        else:
            retstr+="DataSequence genrated from " +self.DNAseqlabel+" with a length of:" +str(len(self.seq))+"\n""\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        return retstr

#### main () I know that there are only 3 returning dictionary entries in supllied FASTA file, so I generate 3 sets of class objects.  

In [6]:
counter = 1
currentStamp = dt.datetime.now()
for name,seq in readMultilineFASTA("SampleData.fa"):
    if counter == 1:
        DNAtest1 = DNASeq(currentStamp,seq,name,"DNA1",counter)
        Protest1 = ProSeq(currentStamp,DNAtest1.transToPro(),"ProfromDNA1")
        Protest1.setFromDNA(DNAtest1.label,counter*counter)
    if counter == 2:
        DNAtest2 = DNASeq(currentStamp,seq,name,"DNA2",counter)
        Protest2 = ProSeq(currentStamp,DNAtest2.convertfromVal(),"ProfromDNA2")
        Protest2.setFromDNA(DNAtest2.label,counter*counter) 
    if counter == 3:
        DNAtest3 = DNASeq(currentStamp,seq,name,"DNA3",counter)
        Protest3 = ProSeq(currentStamp,DNAtest3.convertfromVal(),"ProfromDNA3")
        Protest3.setFromDNA(DNAtest3.label,counter*counter)
    counter+=1

#### Print results from DNAseq objects

In [7]:
print(DNAtest1)
print(DNAtest2)
print(DNAtest3)

I'm DNASequence: DNA1
DataSequence length:2946
______________________________________________
ATGATGCAGTCCGTGGCTCCTGTGTGCAATGTCTGCGGCGAGCAGGTGGGGCTTGGTGCCAATGGGGAGGTTTTCGTGGCATGCCACGAGTGTAATTTCCCCATTTGCAAGGCTTGTTTCGATGAAGATGTCAAGGCTGGGCGTAAAGTTTGCTTGCAGTGTGGTATTCCCTATGACGATAACCCGTTGGCGGAGTATGAAACAAAGGTGTCAGGCACTCGATCCACAATGGAAGCTCACCTGAATAATACACAGGATACAGGAATTCATGCTAGGCATATCAGCAGTGTGTCTACGTTGGATAGTGAATTAAACGATGAATCTGGCAATCCGATTTGGAAGAATAGAGTGGAAAGTTGGAAGGATAAGAAGGATAAGAAGGATAAAAAGATCAAGAAGAAAAAGGATACACCTAATGGGGAAAAAGAGGCTCAAATTCCACCTGAGAAGCAGATGACAGAGGAATATTCATCAGAGGCTGCGGAACCACTTTCAACTCTCGTCCCACTTCCATCTAACAGAATCACACCATACAGAACTGTTATAATTATGCGATTGATCATTCTCGCCCTTTTCTTCCATTATCGAGTAACAAATCCTGTTGATAGTGCTTACGGTCTATGGTTCACTTCGATCATATGTGAGATCTGGTTTGCTTTTTCTTGGGTGTTGGATCAGTTTCCTAAGTGGTCTCCAGTTAATCGGACTACATTTACTGACAGGTTATCTGCCAGGTTTGAAAGAGAGGGTGAACTCTCCGAGCTTGCTGCTGTGGATTTCTTCGTAAGTACAGTTGATCCGTTGAAAGAACCGCCCTTGATTACTGCCAATACCGTGCTTTCTATCCTTGCTGTAGACTACCCTGTGGACAAAGTTTCCTGCTATGTGTCTGATGATGGTGCTGCC

#### Print results from ProSeq objects

In [8]:
print(Protest1)
print(Protest2)
print(Protest3)
#Demo a convert from sequence translation dictionary in parent class
print("-----------------------------\nProSeq convertfromKEY method:  \n-----------------------------\n")
print(Protest3.convertfromKEY())
print("-----------------------------\nProSeq acidCount method:  \n-----------------------------\n")
print(Protest3.countAcids())

I'm Protein Sequence: ProfromDNA1
DataSequence genrated from DNA1 with a length of:982

______________________________________________
MMQSVAPVCNVCGEQVGLGANGEVFVACHECNFPICKACFDEDVKAGRKVCLQCGIPYDDNPLAEYETKVSGTRSTMEAHLNNTQDTGIHARHISSVSTLDSELNDESGNPIWKNRVESWKDKKDKKDKKIKKKKDTPNGEKEAQIPPEKQMTEEYSSEAAEPLSTLVPLPSNRITPYRTVIIMRLIILALFFHYRVTNPVDSAYGLWFTSIICEIWFAFSWVLDQFPKWSPVNRTTFTDRLSARFEREGELSELAAVDFFVSTVDPLKEPPLITANTVLSILAVDYPVDKVSCYVSDDGAAMLTFESLAETSEFATKWVPFCKKFSIEPRAPEFYFSQKIDYLKDKVQPSFVKERRAMKRDYEEFKVRMNALVAKAQKTPEEGWTMQDGTPWPGNNSRDHPGMIQVFLGHSGAYDIEGNELPRLVYVSREKRPGYPHHKKAGAENALVRVSAVLTNAPYILNLDCDHYVNNSQAIREAMCFLMDPQVGREVCYVQFPQRFDGIDRSDRYANRNTVFFDVNMKGLDGIQGPVYVGTGCCFNRQALYGYGPPSMPALSKAASSSSCSCCCPSKKPSKDVSEAYRDAKQEELDAAIFNLRDIENYDELERSMLISQTSFEKTFGLSSVFIESTLMENGGVAESSNPSTLIKEAIHVISCGYEEKTAWGKEIGWIYGSITEDILTGFKMHCRGWRSIYCMPLRPAFKGSAPINLSDRLHQVLRWALGSVEIFLSRHCPLWYGFAGGRLKLLQRMAYINTIVYPFTSLPLVAYCTLPAICLLTGKFIIPTLTNLASALFLGLFISIIATSVLELRWSGVRIEDLWRNEQFWVIGGVSAHLFAVFQGFLKMLAGIDTNFTVTTKSAEDTE

### I added a datetime stamp to seq class, and import index to child classes thinking that might aid in sorting, these attributes were not displayed but were assigned